In [1]:
import pandas as pd
import numpy as np
import pylab as pl
import scipy.io as sio
import seaborn as sns
import statsmodels.api as sm
import funcs as func
import sys
import pdb
import glob

data_dir = "../../Data/Y_maze/Decision_times/"
data_target_dir = "../../Data/processed_data/Y_maze/"


In [2]:
#experiment_type = "NoConfLowVolDTs"



In [3]:
col_names = ["time (ms)","event_marker","current_trial_num_in_block","block","low_rew_%","high_rew_%","low_trial_range(block)","high_trial_range(block)","block#","rewarded_trial_count","total_trial_count","sig","x_pos","y_pos","trip","chnc","current_hall","previous_hall","lick_rate_hall1","lick_rate_hall2","lick_rate_hall3","wtr","mouse_distance","TTL"]

non_action_em = [0,11,12,9,10]
left_action_em = [1,2,3,4]
right_action_em = [5,6,7,8]
rewarded_em = [1,3,5,7]
unrewarded_em = [2,4,6,8]

In [4]:
def read_raw_data(mat):
    # block, 0 = left, 1 = right
    #columns = ["time (ms)", "event_markers", "current_trial_num_in_block","block","reward_probability","conflict","volatility","block_num","rewarded","action_chosen","condition"]
    r,c = np.shape(mat[0])
    raw_df = pd.DataFrame(columns = list(np.arange(0,c,1))+["session"])

    for i in np.arange(len(mat)):
        temp = pd.DataFrame(mat[i])
        temp["session"] = i
        raw_df = raw_df.append(temp)

    return raw_df



def process_raw(raw,experiment_type):
    processed_df = pd.DataFrame(columns=["conflict","condition","volatility","event_marker","block","reward_probability","trial_length_range","session","block_num","trial_num","chosen_action","rewarded"])
    conf = experiment_type.split('Conf')[0]
    #cond = experiment_type.split('Vol')[1]
    cond = "Control" # change when stim data is here !!!!!!
    vol = experiment_type.split('Vol')[0].split('Conf')[1]
    for grp in raw_df.groupby(["session","block#","current_trial_num_in_block"]):
        temp = dict()
        temp["session"] = grp[0][0]
        temp["conflict"] = conf
        temp["volatility"] = vol
        temp["condition"] = cond
        temp["block_num"] = grp[0][1]
        temp["trial_num"] = grp[0][2]
#         time_before_iti = grp[1].loc[grp[1]["event_marker"]!=12]
#         temp["RT(ms)"] = np.max(time_before_iti["time (ms)"]) - np.min(time_before_iti["time (ms)"])
        action_em = list(set(np.unique(grp[1]["event_marker"]))-set(non_action_em))
        #print(action_em)       
        if len(action_em) == 1:
            temp["event_marker"] = action_em[0]
            if action_em[0] in left_action_em:
                temp["chosen_action"] = "left"
            elif action_em[0] in right_action_em:
                temp["chosen_action"] = "right"
            if action_em[0] in rewarded_em:
                temp["rewarded"] = "rewarded"
            elif action_em[0] in unrewarded_em:
                temp["rewarded"] = "unrewarded"
        else:
            #print("no action ems",np.unique(grp[1]["event_marker"]))
            #print(grp[0])
            temp["event_marker"] = np.nan
            temp["reward_probability"] = np.nan
            temp["chosen_action"] = np.nan
            temp["rewarded"] = np.nan

        temp["block"] = "left" if np.unique(grp[1]["block"])[0] == 0 else "right"
        temp["trial_length_range"] = str(np.unique(grp[1]["low_trial_range(block)"])[0])+"-"+str(np.unique(grp[1]["high_trial_range(block)"])[0])

        temp["reward_probability"] = str(np.unique(grp[1]["low_rew_%"])[0])+"-"+str(np.unique(grp[1]["high_rew_%"])[0])
        temp["optimal_action"] = 1 if temp["block"] == temp["chosen_action"] else 0
        processed_df = processed_df.append(temp,ignore_index=True)

    return processed_df



In [5]:
all_conflicts = pd.DataFrame()

exp_files = glob.glob(data_dir+"*.mat")
exp_files
for ef in exp_files:
    print(ef)
    exp_type = ef.split('/')[-1].split('.')[0]
    exp_mat = sio.loadmat(ef)
    raw_df = read_raw_data(exp_mat['overlap'][0])
    # Replace the column numbers with the labels that Julia gave
    raw_df = raw_df.rename(columns={i:cn for i,cn in zip(np.arange(len(col_names)),col_names)})
    processed_df = process_raw(raw_df,exp_type)
    processed_df = processed_df.dropna()
    
    rt_df = read_raw_data(exp_mat['overlap'][1])
    rt_df = rt_df.rename(columns ={0:"RT(ms)"})
    final_df = processed_df.merge(rt_df,on="session")
    final_df.to_csv(data_target_dir+exp_type+"_processed.csv")
    
    all_conflicts = all_conflicts.append(final_df)
    
    

../../Data/Y_maze/Decision_times/NoConfLowVolDTs.mat
../../Data/Y_maze/Decision_times/HighConfHighVolDTs.mat
../../Data/Y_maze/Decision_times/LowConfLowVolDTs.mat
../../Data/Y_maze/Decision_times/NoConfHighVolDTs.mat
../../Data/Y_maze/Decision_times/HighConfHighVolDTs (1).mat
../../Data/Y_maze/Decision_times/LowConfHighVolDTs.mat
../../Data/Y_maze/Decision_times/HighConfLowVolDTs.mat


In [7]:
all_conflicts.to_csv(data_target_dir+"all_experiments_df_with_DTs.csv")


In [ ]:
sns.histplot(x="RT(ms)",data=all_conflicts,hue='')